In [77]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [78]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNDY2NDRjNTctNGY0My00ODg4LWJmMGQtNWYzMTkxMTMyNTc2IiwgInJlY2lwaWVudEtleXMiOiBbIkJBREpZUThMWnc5aUs3Mnp5djFTU2RVaDlxMnVyUzRSamNnVlYxUUZ2TVR6Il0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAibGFiZWwiOiAiTWFpbiAoYWRtaW4pIn0=


In [79]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYjcxYjFhM2UtZThiYi00NGEwLWEzM2EtNGMyOTcxZjU3YzVjIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyJCN3dtdDg0MXpNdGRRQnp4YlA3NFhOcHhhR25NTUVBdXFzWHJxQzY5cG1GayJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==


In [80]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': 'b71b1a3e-e8bb-44a0-a33a-4c2971f57c5c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['B7wmt841zMtdQBzxbP74XNpxaGnMMEAuqsXrqC69pmFk'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [81]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 WMz78hQREYq6Ck7U8acM3xxgR9FaY3GUGc6AnVgDxn3f7ZSWVXdwGiB4TxepYq7FSFqvqQ4t46ZbWdTDH5bgnKn

Public key: 
 gMUbBNJqaTgEPZSZeUvHBiGzBdceusw6HgNWM9bQgW8


**Sending a connection request to acapy**

In [82]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [83]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [84]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

### FEATURE DISCOVERY

In [85]:
pp.pprint(util.getFeatureDiscovery(agents[0]))

{'@id': '01335790-9ef2-41d6-810d-0fe7f53c0785',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1'},
               {'pid': 'https://didcomm.org/present-proof/1.0'},
               {'pid': 'https://didcomm.org/issue-credential/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0'},
               {'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/basicmessage/1.0'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1'},
               {'pid': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-basicmessage/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-static-connections/0.1'},
               {'pid': 'did:sov:BzCbsNYh

### Notes

* To receive invitation use "new connections" protocol "receive invitation"
* To create invitation use create invitation from "invitations" protocol

### Done
* create invitation
* receive invitation
* basic message
    * get
    * send
    * basic message
    * deletes
* dids
    * list dids
    

### Todo
* basic message sorting
* add problem report for did on ledger
* add problem report for schema send

In [86]:
# message = util.buildMessage(
#         'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/create-invitation',
#         label="ConnectionBetweenAgents",
#         auto_accept='auto',
#         alias="agent1ToAgent2"
#     )
# createInvitation = util.sendMessage(message, agents[0])

In [87]:
# createInvitation

In [88]:
message = util.buildMessage(
        'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/schema-exchange/1.0/send',
        payload="asdasdad"
    )
schemaExchange = util.sendMessage(message, agents[0])

In [89]:
schemaExchange

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0/problem-report',
 '@id': 'a7640df6-0ca1-490f-baf6-21664047a03d',
 '~thread': {'thid': 'be26c7c2-5cb6-4089-9c01-76bff4f626b9'},
 'explain-ltxt': 'Duplicate',
 'who-retries': 'none'}

In [90]:
pp.pprint(util.connectAgents(agents[0], agents[1]))

{'@id': 'd0c43618-aedf-4406-ac12-899a7206ea31',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/connection',
 'connection_id': 'c4d78d86-45d2-4bbb-958a-1565cbb7d724',
 'label': 'ConnectionBetweenAgents',
 'my_did': 'H515yqZjjYWAf7kaGWwdQV',
 'raw_repr': {'accept': 'auto',
              'connection_id': 'c4d78d86-45d2-4bbb-958a-1565cbb7d724',
              'created_at': '2020-06-09 15:26:50.912421Z',
              'initiator': 'external',
              'invitation_key': 'CjoRY7jWu1RndtBGnc1b4WndCCZWBVMRs84nSAFx46wy',
              'invitation_mode': 'once',
              'my_did': 'H515yqZjjYWAf7kaGWwdQV',
              'request_id': '30a8aa44-6944-4dd1-8f33-486436e6158a',
              'routing_state': 'none',
              'state': 'request',
              'their_label': 'ConnectionBetweenAgents',
              'updated_at': '2020-06-09 15:26:50.941542Z'},
 'state': 'pending'}


In [91]:
pp.pprint(util.getConnections(agents[1]))

{'@id': '751096e4-9200-43bb-9277-f3270fb4c5a8',
 '@type': 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/list',
 'connections': [{'connection_id': 'd14627b9-10d5-4680-baef-cc021f16b383',
                  'label': 'Plugin',
                  'my_did': 'Rkb5dEYibxXfDU9HBUwzh5',
                  'raw_repr': {'accept': 'auto',
                               'connection_id': 'd14627b9-10d5-4680-baef-cc021f16b383',
                               'created_at': '2020-06-09 15:15:09.385387Z',
                               'initiator': 'self',
                               'invitation_key': '9R23CGodcwRATB5vhBpxPFvL9uizHDSgt33ZTvQUn97a',
                               'invitation_mode': 'once',
                               'my_did': 'Rkb5dEYibxXfDU9HBUwzh5',
                               'routing_state': 'none',
                               'state': 'response',
                               'their_did': 'M8TjGvQ3bi82pv4WbrpfUv',
                   

In [92]:
newDid = util.createDid(agents[0])
pp.pprint(newDid['result'])

{'did': 'U3PsyA1jSfoFwNEJHWa1Xj',
 'verkey': 'Fjq3NNooQaLBnXdbHGUbdjBNWG5esUhaM6Vcp9xfYySu'}


In [93]:
util.registerDidOnLedger(agents[0], newDid['result']['did'], newDid['result']['verkey'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/response',
 '@id': 'c195e5cb-1fe3-4524-ac21-5fe1ec02efef',
 '~thread': {'thid': '30a8aa44-6944-4dd1-8f33-486436e6158a'},
 'connection~sig': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/signature/1.0/ed25519Sha512_single',
  'signature': 'tnSzI2NeLYL_4VAsgSXAl-EyB5RHuMyQ5pFJBei80APsv1SCxz3OWAZTH3nz2x75J4sHpfqcmkLFgqm8xLvOBQ==',
  'sig_data': 'AAAAAF7fqjt7IkRJRCI6ICJLN0RDbTdwM3pnclJYeG5wc3VXdVEyIiwgIkRJRERvYyI6IHsiQGNvbnRleHQiOiAiaHR0cHM6Ly93M2lkLm9yZy9kaWQvdjEiLCAiaWQiOiAiZGlkOnNvdjpLN0RDbTdwM3pnclJYeG5wc3VXdVEyIiwgInB1YmxpY0tleSI6IFt7ImlkIjogImRpZDpzb3Y6SzdEQ203cDN6Z3JSWHhucHN1V3VRMiMxIiwgInR5cGUiOiAiRWQyNTUxOVZlcmlmaWNhdGlvbktleTIwMTgiLCAiY29udHJvbGxlciI6ICJkaWQ6c292Oks3RENtN3AzemdyUlh4bnBzdVd1UTIiLCAicHVibGljS2V5QmFzZTU4IjogIkFzUURwM05vVldNMVVYMkd2TWJndUNhQnlyWVF3R01QQ2ZVRmFtSDI5N3I0In1dLCAiYXV0aGVudGljYXRpb24iOiBbeyJ0eXBlIjogIkVkMjU1MTlTaWduYXR1cmVBdXRoZW50aWNhdGlvbjIwMTgiLCAicHVibGljS2V5IjogImRpZDpzb3Y6SzdEQ203cDN6Z3J

In [94]:
util.setPublicDid(agents[0], newDid['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '5698695e-d53f-41ee-b58e-a76da758def2',
 '~thread': {'thid': 'b4d7e852-ee2f-4bf5-8219-c3e993713797'},
 'result': {'metadata': {'public': True},
  'verkey': 'Fjq3NNooQaLBnXdbHGUbdjBNWG5esUhaM6Vcp9xfYySu',
  'did': 'U3PsyA1jSfoFwNEJHWa1Xj'}}

In [95]:
pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

{'@id': 'f2ac1175-b7f8-4778-a0ed-63bc665ca401',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1/schema-id',
 'schema_id': "('U3PsyA1jSfoFwNEJHWa1Xj:2:schesssdddvd:1.0', {'ver': '1.0', "
              "'id': 'U3PsyA1jSfoFwNEJHWa1Xj:2:schesssdddvd:1.0', 'name': "
              "'schesssdddvd', 'version': '1.0', 'attrNames': "
              "['memesssaaaaaaa'], 'seqNo': 15})",
 '~thread': {'thid': 'f6cafbbb-9020-4c6c-a5f8-bbf3526ae25f'}}


In [96]:
result = util.schemaGetList(agents[0])
last_schema_id = result['results'][-1]['schema_id']
print(result)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schemas/0.1/schema-list', '@id': 'b67ec8e7-85f3-45a3-93d1-6fefba9a325d', '~thread': {'thid': '86c29bc9-d1b8-4427-a663-888962d72930'}, 'results': [{'schema_version': '1.0', 'updated_at': '2020-06-09 14:56:43.854957Z', 'schema_id': 'U', 'state': 'written', 'author': 'other', 'schema_name': 'schesssdddvd', 'attributes': ['memesssaaaaaaa'], 'created_at': '2020-06-09 14:56:43.854957Z'}, {'schema_version': '1.0', 'updated_at': '2020-06-09 15:26:54.290235Z', 'schema_id': 'U3PsyA1jSfoFwNEJHWa1Xj:2:schesssdddvd:1.0', 'state': 'written', 'author': 'self', 'schema_name': 'schesssdddvd', 'attributes': ['memesssaaaaaaa'], 'created_at': '2020-06-09 15:26:54.290235Z'}, {'schema_version': '1.0', 'updated_at': '2020-06-09 14:56:43.771284Z', 'schema_id': 'UTckeR8yEogf7QUDKNoFH7:2:schesssdddvd:1.0', 'state': 'written', 'author': 'self', 'schema_name': 'schesssdddvd', 'attributes': ['memesssaaaaaaa'], 'created_at': '2020-06-09 14:56:43.771284Z'}, {'sche

In [97]:
util.schemaGet(agents[0], last_schema_id)

KeyboardInterrupt: 

In [ ]:
result = util.sendCredentialDefinition(agents[0], last_schema_id)
cred_def_id = result['cred_def_id']
print(cred_def_id)

In [ ]:
util.getCredentialDefinitionList(agents[0])

In [ ]:
util.getCredentialDefinition(agents[0], cred_def_id)